In [1]:
import datetime
import json
import os
import hashlib
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse
import requests

In [7]:
class FlashBlock:
    
    def __init__(self):
        self.chain = [] 
        self.transactions = []
        self.nodes = set()
        self.addBlock(proof = 1, prevHash = "0")
    
    def addBlock(self, proof, prevHash):
        block = {
            "index": len(self.chain),
            "datetime": str(datetime.datetime.now()),
            "proof": proof, 
            "prevHash": prevHash,
            "transactions": self.transactions
        }
        self.transactions = []
        self.chain.append(block)
        return block
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encodedBlock).hexdigest()
    
    def proofOfWork(self, prevProof):
        new_proof = 1
        solved = False
        
        while solved is False:
            hashProblem = hashlib.sha256(str(new_proof ** 2 - prevProof **2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    
    def isChainValid(self, chain):
        previousBlock = chain[0]
        current_index = 1 
        
        while current_index < len(self.chain):
            current_block = chain[current_index]
            currentBlock_prevHash = current_block["prevHash"]
            if currentBlock_prevHash != hashBlock(previousBlock):
                return False
            currentProof = current_block["proof"]
            previousProof = previousBlock["proof"]
            hashProblem = hashlib.sha256(str(currentProof ** 2 - previousProof **2).encode()).hexdigest()
            if hashProblem[:4] != "0000":
                return False
            previousBlock = current_block
            current_index += 1
        return True
    
    def addTransaction(self, sender, receiver, amount):
        self.transactions.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        prevBlock = self.getLastBlock()
        return len(prevBlock["index"]) + 1
    
    def addNode(self, address):
        nodeIP = urlparse(address)
        self.nodes.add(nodeIP.netloc)
        
    def replaceChain(self):
        network = self.nodes
        longest_chain = None
        max_length = len(self.chain)
        
        for node in network:
            response = requests.get(f"http://{node}/chain")
            ##Few checks
            if response.status_code == 200:
                length = response["length"]
                chain = response["chain"]
                if len(chain) > max_length and self.isChainValid(chain):
                    max_length = length
                    longest_chain = chain
        if longest_chain: #not None 
            self.chain = longest_chain
            return True 
        return  False
            
        